# mdb_to_df

In [1]:
import pyodbc
import os
import pandas as pd
from zipfile import ZipFile


def mdb_to_df(file_name, sql):

    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        fr'DBQ={file_name};'
    )
    print(conn_str)
    cnxn = pyodbc.connect(conn_str)

    df = pd.read_sql(sql, cnxn)

    print(f'{file_name} Loaded')
    return df

# Read SUM

In [2]:
import numpy as np
from datetime import datetime as dt


def read_sum(period):
    usecols_sum = """
    SELECT CDbl(TimeOn) AS TOn, CDbl(TimeOff) AS TOff,
    StationNr, Alarmcode, ID, Parameter
    FROM tblAlarmLog WHERE TimeOff IS NOT NULL
    union
    SELECT CDbl(TimeOn) AS TOn, TimeOff AS TOff,
    StationNr, Alarmcode, ID, Parameter
    FROM tblAlarmLog WHERE TimeOff IS NULL
    """
    
    file_name = f'../DATA/SUM/{period}-sum.mdb'

    alarms = mdb_to_df(file_name=file_name, sql=usecols_sum)

    alarms['TOn'] = sqldate_to_datetime(alarms['TOn'])
    alarms['TOff'] = sqldate_to_datetime(alarms['TOff'])

    alarms.rename(columns={'TOn': 'TimeOn',
                           'TOff': 'TimeOff'}, inplace=True)

    alarms = alarms[alarms.StationNr >= 2307405]

    alarms = alarms[
        alarms.StationNr <= 2307535].reset_index(
        drop=True)

    alarms.dropna(subset=['Alarmcode'], inplace=True)

    alarms.reset_index(drop=True, inplace=True)

    alarms.Alarmcode = alarms.Alarmcode.astype(int)

    return alarms


def sqldate_to_datetime(column):
    
    try:

        column = column.str.replace(',', '.').astype(float)
    except:
        pass

    day_parts = np.modf(column.loc[~column.isna()])

    column = column.fillna(pd.NaT)
    
    column.loc[column.notna()] = (
        dt(1899, 12, 30) +
        day_parts[1].astype('timedelta64[D]', errors='ignore') +
        (day_parts[0] * 86400000).astype('timedelta64[ms]', errors='ignore')
    )

    column = column.fillna(pd.NaT)
    
    
    return column

# Cascade

In [3]:
# Determine alarms real periods
def cascade(df):

    df.reset_index(inplace=True, drop=True)
    df['TimeOffMax'] = df.TimeOff.cummax().shift()

    df.at[0, 'TimeOffMax'] = df.at[0, 'TimeOn']

    return df


# looping through turbines and applying cascade method
def apply_cascade(result_sum):

    # Sort by alarm ID
    result_sum.sort_values(['ID'], inplace=True)
    df = result_sum.groupby('StationId').apply(cascade)

    mask_root = (df.TimeOn.values >= df.TimeOffMax.values)
    mask_children = (df.TimeOn.values < df.TimeOffMax.values) & (
        df.TimeOff.values > df.TimeOffMax.values)
    mask_embedded = (df.TimeOff.values <= df.TimeOffMax.values)

    df.loc[mask_root, 'NewTimeOn'] = df.loc[mask_root, 'TimeOn']
    df.loc[mask_children, 'NewTimeOn'] = df.loc[mask_children, 'TimeOffMax']
    df.loc[mask_embedded, 'NewTimeOn'] = df.loc[mask_embedded, 'TimeOff']

    df.drop(columns=['TimeOffMax'], inplace=True)

    df.reset_index(inplace=True, drop=True)

    TimeOff = df.TimeOff
    NewTimeOn = df.NewTimeOn

    df['RealPeriod'] = abs(TimeOff - NewTimeOn)

    mask_siemens = (df['Error Type'] == 1)
    mask_tarec = (df['Error Type'] == 0)

    df['Period Siemens(s)'] = df[mask_siemens].RealPeriod  # .dt.seconds
    df['Period Tarec(s)'] = df[mask_tarec].RealPeriod  # .dt.seconds
    # df['RealPeriod'] = df['RealPeriod'].dt.seconds

    return df

# Periods

In [4]:
from dateutil.relativedelta import relativedelta
from calendar import month_name, different_locale

def get_month_name(month_no, locale):
    with different_locale(locale):
        return month_name[month_no].capitalize()
    

period = input('period ex: "2020-07"') or '2020-07'

period_dt = dt.strptime(period, "%Y-%m")
period_month = period_dt.month
period_month_name = get_month_name(period_month, 'french')
period_year = period_dt.year

days_in_period = pd.Period(f'{period}').days_in_month

previous_period_dt = period_dt + relativedelta(months=-1)
previous_period = previous_period_dt.strftime("%Y-%m")

lastday_period_dt = period_dt + relativedelta(months=1, days=-1)
lastday_period = lastday_period_dt.strftime("%Y-%m-%d")


lastday_previous_period_dt = period_dt + relativedelta(days=-1)
lastday_previous_period = lastday_previous_period_dt.strftime("%Y-%m-%d")

days_to_previous_period = len(pd.date_range(
    f'{period_year}-01-01', lastday_previous_period_dt))

period ex: "2020-07" 2020-08


# Import last period cumul

In [5]:
df_last_ax1 = pd.read_excel(f'./input/{previous_period}/output_xlsxwriter.xlsx', sheet_name= 'ax1', index_col=0)

last_boost_cumul = df_last_ax1.iat[0, 7]
last_ratio_cumul = df_last_ax1.iat[0, 8]
last_mtbf_cumul = df_last_ax1.iat[0, 9]
last_mttr_cumul = df_last_ax1.iat[0, 10]
last_mtti_cumul = df_last_ax1.iat[0, 11]
last_onee_cumul = df_last_ax1.iat[0, 12]

In [6]:
df_last_ax1

,LTA-Lost Time,Indispo. Total %,Indispo. Tarec %,Indispo. Siemens %,Indispo. ONEE %,Indispo. Ebop %,Pertes élctriques en MWh,Power Boost en MWh,Performance moyenne des turbines,MTBF - Mean Time Between Failure,MTTR - Mean Time To Repair,MTTI - Mean Time To Intervention,Compteurs ONEE MWh
Indicateurs annuels :,0,2.85,0.71,2.14,0,0,14647.00,6387.148547,96.341666,39.283167,11.160979,8.086052,730430.9
Indicateurs du mois 2020-07 :,0,1.79,0.20,1.59,0,0,4271.14,2929.048547,105.122672,48.472215,10.931891,7.875000,0.0


# ONEE

In [7]:
onee_period = float(input('onee_period') or 167482.8)

onee_cumul = (last_onee_cumul + onee_period)

onee_period 


# Power Boost

In [8]:

def read_boost(file_name):
    usecols_cnt = '''TimeStamp, StationId, wtc_BoostKWh_endvalue'''

    sql_cnt = f"Select {usecols_cnt} FROM tblSCTurCount;"

    cnt = mdb_to_df(file_name=file_name, sql=sql_cnt)

    cnt['TimeStamp'] = pd.to_datetime(
        cnt['TimeStamp'], format='%m/%d/%y %H:%M:%S')

    return cnt

def boost_endvalue_check(df):
    df = df.reset_index().sort_values('TimeStamp')
    df.drop('index', axis=1, inplace=True)
    clmn = df['wtc_BoostKWh_endvalue']
    x = (clmn.iat[-1] - clmn.iat[0])/1000
    # df = df.diff().query('wtc_BoostKWh_endvalue < 0')
    if x < 0:
        return 0
    else:
        return x



In [9]:

# last_boost_cumul = input('last_boost_cumul') or 3458.1

boost = read_boost(f'../Availability_Warranty_Dash/monthly_data/uploads/{period}/{period}-cnt.mdb')

boost_period = boost.groupby('StationId').apply(lambda df: boost_endvalue_check(df)).sum()

boost_cumul = last_boost_cumul + boost_period

DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../Availability_Warranty_Dash/monthly_data/uploads/2020-08/2020-08-cnt.mdb;
../Availability_Warranty_Dash/monthly_data/uploads/2020-08/2020-08-cnt.mdb Loaded


In [10]:
boost_period, boost_cumul

(2290.3298710937506, 8677.47841796875)

# df ax 19 Performance ratio 

In [11]:

# df_ax19 = pd.read_html(f'./input/{period}/TAREC REPORTING.html', thousands=' ', decimal=',')[0]

# df_ax19['Turbine'] = df_ax19['Turbine'].str.extract(r'\[[^\d]*(\d+)[^\d]*\]').values.astype(int)

# df_ax19 = df_ax19[['Turbine', 'Ratio']]

# df_ax19.Ratio.mean()

# # last_ratio_cumul = input('last_ performance_ratio_cumul') or 94.846

# ratio_period = df_ax19.Ratio.mean()

# ratio_cumul = (last_ratio_cumul*days_to_previous_period + ratio_period*days_in_period)/(days_to_previous_period+days_in_period)

# df_ax19

# Calcul mois

In [12]:
import pandas as pd

df = pd.read_csv(f'../Availability_Warranty_Dash/monthly_data/results/{period}-Availability.csv',
                    decimal=',', sep=';')
df.columns = ['_'.join(str(v) for v in tup) if type(tup) is tuple else tup for tup in df.columns]
df.TimeStamp = pd.to_datetime(df.TimeStamp)
df.to_pickle(f'../Data/results/{period}.pkl')

In [13]:


alarms = read_sum(period)

results = pd.read_pickle(f'../DATA/results/{period}.pkl')

alarms.rename(columns={'StationNr': 'StationId'}, inplace=True)

alarms['StationId'] = alarms['StationId'] - 2307404
results['StationId'] = results['StationId'] - 2307404



DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2020-08-sum.mdb;
../DATA/SUM/2020-08-sum.mdb Loaded


In [14]:
# abbreviations
Frame = pd.DataFrame
c1 = 'Alarmcode'
c2 = 'Error Group'

error_group = pd.concat([Frame({c1: range(901, 2101), c2: 'System'}),
                         Frame({c1: range(2101, 3000), c2: 'Generator'}),
                         Frame({c1: range(3100, 4000), c2: 'Hub'}),
                         Frame({c1: range(4100, 5000), c2: 'Gear'}),
                         Frame({c1: range(5000, 6000), c2: 'Grid'}),
                         Frame({c1: range(6100, 7000), c2: 'Rotor'}),
                         Frame({c1: range(7100, 8000), c2: 'Hydraulics'}),
                         Frame({c1: range(8000, 8400), c2: 'Environement'}),
                         Frame({c1: range(8450, 9000),
                                c2: 'Turbine cond...'}),
                         Frame({c1: range(9100, 10000), c2: 'Brake'}),
                         Frame({c1: range(10100, 11000), c2: 'Yaw'}),
                         Frame({c1: range(11100, 12000), c2: 'PFC'}),
                         Frame({c1: range(12100, 13000), c2: 'Transformer'}),
                         Frame({c1: range(13000, 14000), c2: 'Converter-1'}),
                         Frame({c1: range(14000, 15000), c2: 'Gen.inverter'}),
                         Frame({c1: range(15000, 16000), c2: 'Grid inverter'}),
                         Frame({c1: range(16000, 17000), c2: 'Main bearing'}),
                         Frame({c1: range(17000, 18300), c2: 'Converter-2'}),
                         Frame({c1: range(62001, 64000), c2: 'Controller'}),
                         Frame({c1: range(64000, 65200), c2: 'MISCELLANEOUS'})])


In [15]:
reorder = ['System',
           'Generator',
           'Hub',
           'Gear',
           'Grid',
           'Rotor',
           'Hydraulics',
           'Environement',
           'Turbine cond...',
           'Brake',
           'Yaw',
           'PFC',
           'Transformer',
           'Converter-1',
           'Gen.inverter',
           'Grid inverter',
           'Main bearing',
           'Converter-2',
           'Controller',
           'MISCELLANEOUS']

In [16]:
error_list = pd.read_excel(
    r'Error_Type_List_Las_Update_151209.xlsx',
    usecols=lambda x: x != 'Type Selected')

error_list.Alarmcode = error_list.Alarmcode.astype(int)  # ,errors='ignore'

error_list.drop_duplicates(subset=['Alarmcode'], inplace=True)

error_list = error_list.merge(error_group, on='Alarmcode', how='left')

# ------------------------------------------------------------------------
''' label scada alarms with coresponding error type
and only keep alarm codes in error list'''
result_sum = pd.merge(alarms, error_list,
                        on='Alarmcode',
                        how='inner', sort=False)

# Remove warnings
result_sum = result_sum.loc[result_sum['Error Type'].isin([1, 0])]

# apply cascade
alarms_result_sum = apply_cascade(result_sum)

# only keep  parent alarms
parent_result_sum = alarms_result_sum.query('TimeOn == NewTimeOn')

# dash duree
main_result_sum = alarms_result_sum.query('RealPeriod > @pd.Timedelta(0)')
# main_result_sum['ALL duree'] = main_result_sum['TimeOff'] -main_result_sum['TimeOn']

In [17]:
alarms

,TimeOn,TimeOff,StationId,Alarmcode,ID,Parameter
0,2020-08-01 00:00:34.040,NaT,85,67,3088903,3.0 ...
1,2020-08-01 00:00:34.050,NaT,85,68,3088904,478 ...
2,2020-08-01 00:00:34.059,NaT,85,69,3088905,21.0 ...
3,2020-08-01 00:00:34.069,NaT,85,61,3088906,Out Of Range ...
4,2020-08-01 00:01:15.779,2020-08-01 00:20:12.710,4,50950,3088907,...
...,...,...,...,...,...,...
74778,2020-08-31 23:57:42.300,2020-08-31 23:59:57.869,85,50950,3164889,...
74779,2020-08-31 23:58:32.000,2020-09-01 02:02:02.000,15,63004,3167101,...
74780,2020-08-31 23:58:34.000,2020-09-01 01:59:27.000,15,63003,3167102,...
74781,2020-08-31 23:58:34.060,NaT,15,9,3167105,0 ...


## MTBF MTTR 

In [18]:
manual_stop = [1001,
               1002,
               1005,
               1007,
               1008,
               1009,
               1015,
               1016,
               1017,
               1021,
               1025,
               1026,
               1027,
               8216,
               8217,
               8222,
               ]

In [19]:
query = '(Alarmcode not in @manual_stop) & (RealPeriod >= @pd.Timedelta("2H"))'

freq_period = len(main_result_sum.query(query))

duree_period = (main_result_sum.query(query)
                               .RealPeriod
                               .sum().total_seconds()
                               / 3600)

mtbf_period = (days_in_period * 24 * 131 - duree_period) / freq_period / 24

mtbf_period

24.639150192072822

In [20]:
mttr_period = duree_period/freq_period

mttr_period

6.599045697000681

In [21]:
# Generate the new calculated MTTBF and MTTR for each error group

MTBF_MTTR_cat = (main_result_sum.query(query).groupby('Error Group')
                                .agg(duree=('RealPeriod', lambda x: x.sum().total_seconds() / 3600),
                                     freq=('TimeOn', 'count'),
                                     )
                                .rename_axis('Catégorie'))

try:
    MTBF_MTTR_cat.loc['Converter-1'] = MTBF_MTTR_cat.loc[['Converter-1', 'Converter']].sum()

    MTBF_MTTR_cat = (MTBF_MTTR_cat_cumul.drop('Converter')
                                        .rename(index={'Converter-1': 'Converter'}))
except:
    pass

temps_total_period = days_in_period * 24 * 131

duree_period = MTBF_MTTR_cat['duree']
freq_period = MTBF_MTTR_cat['freq']


MTBF_MTTR_cat['MTTR en h'] = duree_period/freq_period

MTBF_MTTR_cat['MTBF en j'] = (
    temps_total_period - duree_period) / (freq_period * 24)


## MTTI

In [22]:

def mtti(df, turbine):

    # df = main_result_sum.query('StationId == @turbine')
    df = (df.loc[(df.TimeOn == df.NewTimeOn).shift(-1).fillna(True)]
            .query('TimeOn != NewTimeOn').assign(root=False)
            .append(df.query('TimeOn == NewTimeOn').assign(root=True))
            .sort_values('ID'))

    df.loc[(df.root == False).shift(-1).fillna(False), 'TimeOff'] = df.loc[(df.root == False)]['TimeOff'].values
    df['RealPeriod'] = df['TimeOff'] - df['TimeOn']

    df.drop(df.loc[df.root == False].index, inplace=True)

    df2 = alarms.query('StationId == @turbine & Alarmcode == 50000')

    df = (df2.append(df).sort_values('TimeOn').reset_index(drop=True))

    df = df.query('Alarmcode not in @manual_stop') 

    df = df.loc[(df['Alarmcode']).shift() != (df['Alarmcode'])]

    df = cascade(df)

    mask_root = (df.TimeOn.values >= df.TimeOffMax.values)
    mask_children = (df.TimeOn.values < df.TimeOffMax.values) & (
        df.TimeOff.values > df.TimeOffMax.values)
    mask_embedded = (df.TimeOff.values <= df.TimeOffMax.values)

    df.loc[mask_root, 'NewTimeOn'] = df.loc[mask_root, 'TimeOn']
    df.loc[mask_children, 'NewTimeOn'] = df.loc[mask_children, 'TimeOffMax']
    df.loc[mask_embedded, 'NewTimeOn'] = df.loc[mask_embedded, 'TimeOff']

    
    df.drop(df.query('TimeOn == NewTimeOn & Alarmcode == 50000').index, inplace=True)

    mask = (df.Alarmcode == 50000).shift(-1, fill_value=False)

    df.loc[mask, 'TimeTI'] = df.loc[(df.Alarmcode == 50000), 'TimeOn'].values

    df.loc[mask, 'TimeTI'] = df.loc[mask, 'TimeTI'] - df.loc[mask, 'TimeOn']

    df['TimeTI'] = df['TimeTI'].astype('timedelta64[ns]')


    return df


In [23]:
mtti_df = main_result_sum.groupby('StationId').apply(lambda x: mtti(x, x.name)).reset_index(drop=True)
mtti_df

,TimeOn,TimeOff,StationId,Alarmcode,ID,Parameter,UK Text,Error Type,Error Group,NewTimeOn,RealPeriod,Period Siemens(s),Period Tarec(s),root,TimeOffMax,TimeTI
0,2020-08-06 03:25:55.000,2020-08-06 03:29:24.999,1,3130,3100607,...,Pitch lubrication,0,Hub,2020-08-06 03:25:55.000,0 days 00:03:29.999000,NaT,0 days 00:03:29.999000,True,2020-08-06 03:25:55.000,NaT
1,2020-08-31 23:16:10.000,2020-09-01 00:57:13.000,1,13902,3163693,...,"Converter tripped, Auto Start",1,Converter-1,2020-08-31 23:16:10.000,0 days 01:41:03,0 days 00:00:04,NaT,True,2020-08-06 03:29:24.999,NaT
2,2020-08-06 03:26:54.000,2020-08-06 03:30:23.999,2,3130,3100614,...,Pitch lubrication,0,Hub,2020-08-06 03:26:54.000,0 days 00:03:29.999000,NaT,0 days 00:03:29.999000,True,2020-08-06 03:26:54.000,NaT
3,2020-08-31 23:16:10.000,2020-09-01 00:58:13.999,2,13902,3163879,...,"Converter tripped, Auto Start",1,Converter-1,2020-08-31 23:16:10.000,0 days 01:42:03.999000,0 days 00:00:02.999000,NaT,True,2020-08-06 03:30:23.999,NaT
4,2020-08-06 01:47:43.000,2020-08-06 01:51:13.000,3,3130,3100230,...,Pitch lubrication,0,Hub,2020-08-06 01:47:43.000,0 days 00:03:30,NaT,0 days 00:03:30,True,2020-08-06 01:47:43.000,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,2020-08-16 05:34:41.000,2020-08-16 05:38:10.999,130,3130,3117922,...,Pitch lubrication,0,Hub,2020-08-16 05:34:41.000,0 days 00:03:29.999000,NaT,0 days 00:03:29.999000,True,2020-08-11 15:38:50.000,NaT
770,2020-08-31 16:08:12.999,2020-08-31 16:30:08.000,130,8155,3158217,...,Pri.Windspeed Range error,1,Environement,2020-08-31 16:08:12.999,0 days 00:21:55.001000,0 days 00:21:55.001000,NaT,True,2020-08-16 05:38:10.999,NaT
771,2020-08-31 22:14:55.999,2020-08-31 22:18:26.000,130,3130,3163106,...,Pitch lubrication,0,Hub,2020-08-31 22:14:55.999,0 days 00:03:30.001000,NaT,0 days 00:03:30.001000,True,2020-08-31 16:30:08.000,NaT
772,2020-08-02 22:21:49.000,2020-08-03 01:12:12.000,131,13902,3092981,...,"Converter tripped, Auto Start",1,Converter-1,2020-08-02 22:21:49.000,0 days 02:50:23,0 days 00:00:03.999000,NaT,True,2020-08-02 22:21:49.000,NaT


In [24]:
mtti_period = (mtti_df['TimeTI'].sum() / len(mtti_df.query('Alarmcode == 50000'))).total_seconds()/3600

mtti_period

5.980510391944445

## Graphe 3 ax5

In [25]:
df_ax5 = (main_result_sum.groupby('Error Group')
                  .agg(Freq=('Alarmcode', 'count'),
                       Durée=('RealPeriod', lambda x: x.sum().total_seconds()/3600))
                  .reindex(reorder)
                  .dropna()
                  .reset_index()
                  )

# df_ax5.plot(kind='bar', x='Error Group', ax=ax5)
df_ax5

,Error Group,Freq,Durée
0,System,406.0,538.132224
1,Generator,17.0,14.600000
2,Hub,649.0,165.998608
3,Gear,51.0,149.658612
4,Grid,172.0,229.364172
5,Rotor,14.0,20.085279
6,Hydraulics,42.0,51.840553
7,Environement,22.0,21.005278
8,Brake,36.0,27.922779
9,Yaw,246.0,274.385002


## Graphe 4 ax6

In [26]:
df_ax6 = (main_result_sum.groupby('Alarmcode')
                .agg(Freq=('Alarmcode', 'count'),
                     Durée=('RealPeriod',
                               lambda x: x.sum().total_seconds()/3600))
                .sort_values('Durée', ascending=False)
                .head(20)
                .reset_index()
                .sort_values('Durée', ascending=False))

# df_ax6.plot(kind='bar', x='Alarmcode', ax=ax6)
df_ax6

,Alarmcode,Freq,Durée
0,1001,225,445.754999
1,5104,87,211.717506
2,10100,51,173.670279
3,13902,178,129.113865
4,4104,18,94.388056
5,10109,51,71.231666
6,3145,11,57.336666
7,64066,8,43.089445
8,1018,100,37.664168
9,3130,579,37.422220


## ax7

In [27]:
most_categories = (main_result_sum.groupby('Error Group')
                                  .agg({'RealPeriod': np.sum})
                                  .sort_values('RealPeriod', ascending=False)
                                  .index.values)

realperiod_sum = main_result_sum.RealPeriod.sum()
percentage_per_category = (main_result_sum.groupby('Error Group')
                                  .agg(percentage=('RealPeriod',
                                                   lambda x: 100 * x.sum() / realperiod_sum))
                                  .sort_values('percentage', ascending=False))


df_most_categories = main_result_sum.loc[main_result_sum['Error Group'].isin(most_categories)].copy()

df_most_categories['Error Group'] = df_most_categories['Error Group'].astype("category")

df_most_categories['Error Group'].cat.set_categories(most_categories, inplace=True)

df_most_categories.sort_values('Error Group', inplace=True)


# ------------------------------------------------------------------------------------------------
def extract_important_alarms(df):

    df = df.groupby('Alarmcode').agg({'RealPeriod': np.sum}).reset_index()
    
    df['percentage'] = 100 * df['RealPeriod'] / df['RealPeriod'].sum()
    df.sort_values('percentage', ascending=False, inplace=True)
    df['percentage roll'] = df['percentage'].cumsum()
    df.set_index('Alarmcode', inplace=True)


#     for i in range(1,100):
#         if len(df.query('`percentage roll` < @i')) < 3:
#             continue
#         else:
#             df = df.query('`percentage roll` < @i')
#         break

    return df

# ------------------------------------------------------------------------------------------------

df_ax7 = (df_most_categories.groupby('Error Group', sort=False)
                            .apply(lambda x: extract_important_alarms(x))
                            .reset_index()
                            .merge(error_list[['UK Text', 'Alarmcode']], how='left')
         )

# df_ax7 = (df_most_categories.groupby('Error Group')
#                              .apply(lambda x: (x.groupby('Alarmcode')
#                                                 .agg({'RealPeriod': np.sum})
#                                                 .sort_values('RealPeriod')
#                                                 .tail(4)))
#                              .reset_index()
#                              .merge(error_list[['UK Text', 'Alarmcode']]))

# ------------------------------------------------------------------------------------------------
ax7 = ''
for cat in most_categories:
    
    str1_ax7 = ''
    
    for i, row in df_ax7.query('`Error Group` == @cat').iterrows():
        
        Alarmcode = row['Alarmcode']
        UK_Text = row['UK Text']
        Percentage = round(row['percentage'], 1)
                       
        str1_ax7 += f'\t-\"{Alarmcode} - {UK_Text} - {Percentage}% \".\n'
        
    
    str2_ax7 = f"""La catégorie {cat} - {round(percentage_per_category.loc[cat, 'percentage'], 2)}%, les alarmes les plus importantes sont:
{str1_ax7}
    """
    ax7 += str2_ax7
    
worst_mtbf_cat = MTBF_MTTR_cat['MTBF en j'].idxmin()
worst_mtbf = int(MTBF_MTTR_cat['MTBF en j'].min())

ax7 += f'Le MTBF le moins bon du mois est celui de la catégorie "{worst_mtbf_cat}" :  {worst_mtbf} jours '


In [28]:
with open("Output.txt", "w") as text_file:
    text_file.write(ax7)

## Graph 6 ax9

In [29]:
df_ax9 = pd.merge(
        (results.groupby('StationId')
                       .agg(**{'Durée alarmes': ('Duration 115(s)', lambda x: x.sum()/3600),
                               'Durée autres': ('Duration 20-25(s)', lambda x: x.sum()/3600)})
                       .sort_values('Durée alarmes', ascending=False)
                 # .head(25)
                 .reset_index()
         ),
         (alarms.groupby('StationId')
                      .agg(**{'Freq alarmes': ('Alarmcode', lambda x: int(x[x == 115].count()/2)),
                              'Freq autres': ('Alarmcode', lambda x: x[x == 20].count())})
                      .reset_index()
         ),
         on='StationId'
        ).sort_values('Durée alarmes', ascending=False).head(20)


df_ax9['Duration_20_25 reel'] = df_ax9['Durée autres']
df_ax9['Durée autres'] = df_ax9['Duration_20_25 reel'] - df_ax9['Durée alarmes']
# df_ax9.plot(kind='bar', x='StationId', ax=ax9)

df_ax9

,StationId,Durée alarmes,Durée autres,Freq alarmes,Freq autres,Duration_20_25 reel
0,83,263.856111,0.151389,4,4,264.007500
1,7,118.671944,11.066667,30,30,129.738611
2,52,66.861667,4.476389,24,12,71.338056
3,66,64.722222,16.620000,23,19,81.342222
4,39,61.663333,11.924167,48,54,73.587500
5,92,57.570000,16.106111,40,20,73.676111
6,49,53.186389,9.734167,15,19,62.920556
7,19,47.688889,4.085278,19,17,51.774167
8,113,44.790278,11.235833,32,19,56.026111
9,118,44.623333,6.630833,12,13,51.254167


## Graphe 7 ax10

In [30]:
df_ax10 = (results[['StationId','ELNX', 'EL_indefini_left']]
                .groupby('StationId')
                .sum()
                .sort_values('ELNX', ascending=False)
                .head(20)
                .reset_index())

df_ax10 = round(df_ax10, 2).abs().head(20)

df_ax10

,StationId,ELNX,EL_indefini_left
0,7.0,250480.47,0.00
1,39.0,123493.82,73.87
2,66.0,116291.68,161.48
3,49.0,114044.76,0.00
4,52.0,90080.53,0.00
5,19.0,88520.44,0.00
6,6.0,79491.49,0.00
7,83.0,69586.15,0.00
8,97.0,69541.15,454.77
9,98.0,66180.04,278.66


## ax11

In [31]:
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

period_month_name_ascii = remove_accents(period_month_name).upper()

In [32]:
parts_replaced = pd.read_excel(f'./input/{period}/PART REPLACED SIEMENS SERVICE {period_month_name_ascii} {period_year}.xlsx', skiprows=6)

df_ax11_last = pd.read_excel(f'./results/{previous_period}/TB Performance {previous_period}.xlsx', sheet_name='Feuil3', skiprows=1,
                       usecols='C:E')



parts_replaced = parts_replaced.rename(columns={'REFERENCE': 'item', 'QTE SORTIE': period_month_name})

parts_replaced = parts_replaced.groupby('item').sum().reset_index()[['item', period_month_name]]

df_ax11 = df_ax11_last.merge(parts_replaced, how='outer')[['item', 'Year', period_month_name]]

df_ax11['Year'] = (df_ax11['Year'].fillna(0) + df_ax11[period_month_name].fillna(0))

df_ax11 = df_ax11.fillna(0).astype(int, errors='ignore')

In [33]:
df_ax11.head()

,item,Year,Août
0,A9B10055151,290,28
1,A9B00573133,95,6
2,A9B10027591,43,1
3,A9B00075055,59,5
4,A9B00019620,32,4


## ax12

In [34]:
df_ax12 = pd.read_excel(f'./input/{period}/PART REPLACED SIEMENS SERVICE AOUT 2020.xlsx', skiprows=6)

df_ax12 = (df_ax12.groupby('DESIGNATION')
                  .agg({'QTE SORTIE': np.sum,
                        'N° TURBINE': lambda x: x.str.lstrip('T0').str.cat(sep=', ')})
                  .reset_index())

df_ax12.sort_values('QTE SORTIE').reset_index(drop=True)

,DESIGNATION,QTE SORTIE,N° TURBINE
0,BRAKE F YAW,1,54
1,BREAKE PAD,1,19
2,TRIP UNIT,1,49
3,COOLER MOTOR,1,6
4,STIC BOX,1,43
5,RADIATOR,1,7
6,MAIN BREAKER,1,49
7,INLINE PUMP,2,"7, 92"
8,DC FUSE,2,43
9,DELTA MODULE,2,43


In [35]:
df_ax12 = pd.read_excel(f'./input/{period}/PART REPLACED SIEMENS SERVICE AOUT 2020.xlsx', skiprows=6)

df_ax12 = df_ax12.groupby(['DESIGNATION', 'N° TURBINE']).sum().sort_values(['DESIGNATION', 'QTE SORTIE'])

df_ax12

QTE SORTIE  STOCK YAW
DESIGNATION     N° TURBINE                       
ANEMOMETRE      T100                 2        0.0
                T113                 2        0.0
                T51                  2        0.0
                T52                  2        0.0
BATTERY PACK UP T115                 1        0.0
...                                ...        ...
Yaw Motor       T39                  5      212.0
                T52                  6      204.0
smps master     T118                 1        0.0
                T67                  1        0.0
                T43                  2        0.0

[73 rows x 2 columns]

## table 1 ax1 row 2

In [36]:

df_ax1 = pd.DataFrame(columns=['LTA-Lost Time', 'Indispo. Total %','Indispo. Tarec %', 'Indispo. Siemens %', 'Indispo. ONEE %', 'Indispo. Ebop %',
                               'Pertes élctriques en MWh', 'Power Boost en MWh', 'Performance moyenne des turbines',
                               'MTBF - Mean Time Between Failure', 'MTTR - Mean Time To Repair', 'MTTI - Mean Time To Intervention',
                               'Compteurs ONEE MWh'],
                     index=['Indicateurs annuels :', f'Indicateurs du mois {period} :'])
df_ax1 = df_ax1.fillna(0.)

In [37]:
df_ax1.iat[1, 1] = round(100*results['RealPeriod'].sum()/3600/24/131/31,2)
df_ax1.iat[1, 2] = round(100*results['Period 0(s)'].sum()/3600/24/131/31,2)
df_ax1.iat[1, 3] = round(100*results['Period 1(s)'].sum()/3600/24/131/31,2)

df_ax1.iat[1, 6] = round(results.wtc_ActPower_mean.sum()/6/1000 - onee_period,2)

df_ax1.iat[1, 7] = boost_period

#df_ax1.iat[1, 8] = ratio_period
#
df_ax1.iat[1, 9]  = mtbf_period
df_ax1.iat[1, 10] = mttr_period
df_ax1.iat[1, 11] = mtti_period
df_ax1.iat[1, 12] = onee_period

In [38]:
df_ax1

,LTA-Lost Time,Indispo. Total %,Indispo. Tarec %,Indispo. Siemens %,Indispo. ONEE %,Indispo. Ebop %,Pertes élctriques en MWh,Power Boost en MWh,Performance moyenne des turbines,MTBF - Mean Time Between Failure,MTTR - Mean Time To Repair,MTTI - Mean Time To Intervention,Compteurs ONEE MWh
Indicateurs annuels :,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.000000,0.0,0.00000,0.000000,0.00000,0.0
Indicateurs du mois 2020-08 :,0.0,1.76,0.33,1.43,0.0,0.0,4022.39,2290.329871,0.0,24.63915,6.599046,5.98051,167482.8


## table 2 ax2

In [39]:

df_ax2_header = ['', '', '', 'Travaux de maintenance']
df_ax2 = pd.DataFrame(columns=['Transformateur', 'Serial Defect', 'Corrosion', 'Inspection Fin de garantie',
                               'FSA', 'Assurance', 'Conformité DNSSI', 'Main Bearing', 'BAX', 'Darwin',
                               'Procédure de securité', 'SMI', 'Exploitation', 'Mesures', 'Audit ENGIE'],
                     index=['Etape terminée', 'Etape en cours'])

In [40]:
df_ax2

,Transformateur,Serial Defect,Corrosion,Inspection Fin de garantie,FSA,Assurance,Conformité DNSSI,Main Bearing,BAX,Darwin,Procédure de securité,SMI,Exploitation,Mesures,Audit ENGIE
Etape terminée,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Etape en cours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Calcul Cumul

In [41]:
cumul_alarms = pd.DataFrame()
cumul_results = pd.DataFrame()

for month in range(1, period_month+1):

    month = str(month)
    alarms = read_sum(f'2020-{month.zfill(2)}')

    cumul_alarms = pd.concat([cumul_alarms, alarms])

    # -------------------------------------------------------------------------
    results = pd.read_pickle(f'../DATA/results/2020-{month.zfill(2)}.pkl')
    # results = results[['StationId', 'ELNX', 'Duration 115(s)', 'Duration 20-25(s)',
    #                    'Period 0(s)', 'Period 1(s)', 'RealPeriod',
    #                    'EL_indefini_left']]

    cumul_results = pd.concat([cumul_results, results])


DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2020-01-sum.mdb;
../DATA/SUM/2020-01-sum.mdb Loaded
DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2020-02-sum.mdb;
../DATA/SUM/2020-02-sum.mdb Loaded
DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2020-03-sum.mdb;
../DATA/SUM/2020-03-sum.mdb Loaded
DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2020-04-sum.mdb;
../DATA/SUM/2020-04-sum.mdb Loaded
DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2020-05-sum.mdb;
../DATA/SUM/2020-05-sum.mdb Loaded
DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2020-06-sum.mdb;
../DATA/SUM/2020-06-sum.mdb Loaded
DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2020-07-sum.mdb;
../DATA/SUM/2020-07-sum.mdb Loaded
DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2020-08-sum.mdb;
../DATA/SUM/2020-08-sum.mdb Loaded


In [42]:
cumul_alarms.rename(columns={'StationNr': 'StationId'}, inplace=True)

# only run once otherwise negative turbine numbers
cumul_alarms['StationId'] = cumul_alarms['StationId'] - 2307404
cumul_results['StationId'] = cumul_results['StationId'] - 2307404

In [43]:
cumul_result_sum = pd.merge(cumul_alarms, error_list[[
                        'Alarmcode', 'Error Type', 'Error Group']],
                        on='Alarmcode',
                        how='inner', sort=False)

# Remove warnings
cumul_result_sum = cumul_result_sum.loc[cumul_result_sum['Error Type'].isin([1, 0])]

# apply cascade
cumul_alarms_result_sum = apply_cascade(cumul_result_sum)

#only keep  parent alarms
cumul_parent_result_sum = cumul_alarms_result_sum.query('TimeOn	 == NewTimeOn')
  
cumul_main_result_sum = cumul_alarms_result_sum.query('RealPeriod > @pd.Timedelta(0)')

## MTTI MTTR MTBF cumulé

In [44]:

mtbf_cumul = (last_mtbf_cumul*days_to_previous_period + mtbf_period *
              days_in_period)/(days_to_previous_period+days_in_period)

mttr_cumul = (last_mttr_cumul*days_to_previous_period + mttr_period *
              days_in_period)/(days_to_previous_period+days_in_period)

mtti_cumul = (last_mtti_cumul*days_to_previous_period + mtti_period *
              days_in_period)/(days_to_previous_period+days_in_period)

## Table 2 ax1 row 1

In [45]:
nbr_jrs_total = len(pd.date_range(f'{period_year}-01-01', f'{lastday_period}', freq='D'))

df_ax1.iat[0, 1] = round(100*cumul_results['RealPeriod'].sum()/3600/24/131/nbr_jrs_total,2)
df_ax1.iat[0, 2] = round(100*cumul_results['Period 0(s)'].sum()/3600/24/131/nbr_jrs_total,2)
df_ax1.iat[0, 3] = round(100*cumul_results['Period 1(s)'].sum()/3600/24/131/nbr_jrs_total,2)

df_ax1.iat[0, 6] = round(cumul_results.wtc_ActPower_mean.sum()/6/1000 - onee_cumul,2)

df_ax1.iat[0, 7] = boost_cumul

# df_ax1.iat[0, 8] = ratio_cumul
# 
df_ax1.iat[0, 9] =  mtbf_cumul
df_ax1.iat[0, 10] = mttr_cumul
df_ax1.iat[0, 11] = mtti_cumul
df_ax1.iat[0, 12] = onee_cumul



In [46]:
df_ax1

,LTA-Lost Time,Indispo. Total %,Indispo. Tarec %,Indispo. Siemens %,Indispo. ONEE %,Indispo. Ebop %,Pertes élctriques en MWh,Power Boost en MWh,Performance moyenne des turbines,MTBF - Mean Time Between Failure,MTTR - Mean Time To Repair,MTTI - Mean Time To Intervention,Compteurs ONEE MWh
Indicateurs annuels :,0.0,2.71,0.66,2.05,0.0,0.0,18669.88,8677.478418,0.0,37.422657,10.581389,7.818544,897913.7
Indicateurs du mois 2020-08 :,0.0,1.76,0.33,1.43,0.0,0.0,4022.39,2290.329871,0.0,24.639150,6.599046,5.980510,167482.8


## Graphe 1 ax3

In [47]:
df_ax3 = (cumul_main_result_sum.groupby('Error Group')
                .agg(Freq=('Alarmcode', 'count'),
                     Durée=('RealPeriod', lambda x: x.sum().total_seconds()/3600))
                .sort_values('Freq', ascending=False)
                .reindex(reorder)
                .dropna()
                .reset_index()
                )

# df_ax3.plot(kind='bar', x='Error Group', ax=ax3)
df_ax3

,Error Group,Freq,Durée
0,System,4002.0,8362.616059
1,Generator,51.0,64.565556
2,Hub,5863.0,901.845532
3,Gear,216.0,705.815590
4,Grid,624.0,621.185830
5,Rotor,73.0,133.184998
6,Hydraulics,1176.0,1517.883635
7,Environement,329.0,240.523893
8,Brake,433.0,251.257371
9,Yaw,5566.0,4428.225563


## Graphe 2 ax4

In [48]:
MTBF_MTTR_cat = (main_result_sum.query(query).groupby('Error Group')
                                .agg(duree=('RealPeriod', lambda x: x.sum().total_seconds() / 3600),
                                     freq=('TimeOn', 'count'),
                                     )
                                .rename_axis('Catégorie'))

In [49]:
MTBF_MTTR_cat

,duree,freq
Catégorie,,
Brake,24.426667,2
Controller,74.817778,10
Converter-1,164.277776,35
Environement,14.952778,3
Gear,135.285834,16
Generator,8.445278,1
Grid,146.824170,39
Hub,117.595000,11
Hydraulics,48.669166,10


In [50]:
MTBF_MTTR_cat_prev_cumul = pd.read_excel(
    f'../TB Performance/input/{previous_period}/output_xlsxwriter.xlsx', sheet_name='MTBF & MTTR cumul')

MTBF_MTTR_cat_prev_cumul.dropna(
    how='all', subset=['MTBF en j', 'MTTR en h'], inplace=True)

MTBF_MTTR_cat_prev_cumul.rename(
    columns={'Catégorie Alarme': 'Catégorie'}, inplace=True)

MTBF_MTTR_cat_prev_cumul.set_index('Catégorie', inplace=True)

MTBF_MTTR_cat_prev_cumul

,duree,freq,MTTR en h,MTBF en j
Catégorie,,,,
Brake,395.283889,39,10.135484,715.039227
Controller,1885.333056,160,11.783332,173.902778
Converter,1383.825278,115,12.033263,242.133397
Gear,612.430000,54,11.341296,516.249668
Grid,1557.721111,173,9.004168,160.913844
Hub,506.516389,44,11.511736,633.679435
Hydraulics,1989.331111,195,10.201698,142.667237
Miscellaneous,30.053611,2,15.026806,13950.873883
Rotor,162.034167,20,8.101708,1394.812429


In [51]:
MTBF_MTTR_cat_cumul = MTBF_MTTR_cat.merge(MTBF_MTTR_cat_prev_cumul, on='Catégorie', how='outer').fillna(0)

MTBF_MTTR_cat_cumul.loc['Converter-1'] = MTBF_MTTR_cat_cumul.loc[['Converter-1', 'Converter']].sum()

MTBF_MTTR_cat_cumul = (MTBF_MTTR_cat_cumul.drop('Converter')
                                          .rename(index={'Converter-1': 'Converter'}))

MTBF_MTTR_cat_cumul

,duree_x,freq_x,duree_y,freq_y,MTTR en h,MTBF en j
Catégorie,,,,,,
Brake,24.426667,2.0,395.283889,39.0,10.135484,715.039227
Controller,74.817778,10.0,1885.333056,160.0,11.783332,173.902778
Converter,164.277776,35.0,1383.825278,115.0,12.033263,242.133397
Environement,14.952778,3.0,0.000000,0.0,0.000000,0.000000
Gear,135.285834,16.0,612.430000,54.0,11.341296,516.249668
Generator,8.445278,1.0,307.007778,9.0,34.111975,3098.912001
Grid,146.824170,39.0,1557.721111,173.0,9.004168,160.913844
Hub,117.595000,11.0,506.516389,44.0,11.511736,633.679435
Hydraulics,48.669166,10.0,1989.331111,195.0,10.201698,142.667237


In [52]:
# Generate the new calculated MTTBF and MTTR for each error group

temps_total_cumul = len(pd.date_range(
    f'{period_year}-01-01', f'{lastday_period}', freq='D'))*24*131

MTBF_MTTR_cat_cumul['duree_y'] = MTBF_MTTR_cat_cumul['duree_x'] + MTBF_MTTR_cat_cumul['duree_y']

duree_cumul = MTBF_MTTR_cat_cumul['duree_y']

MTBF_MTTR_cat_cumul['freq_y'] = MTBF_MTTR_cat_cumul['freq_x'] + MTBF_MTTR_cat_cumul['freq_y']

freq_cumul = MTBF_MTTR_cat_cumul['freq_y']

MTBF_MTTR_cat_cumul['MTTR en h'] = duree_cumul/freq_cumul

MTBF_MTTR_cat_cumul['MTBF en j'] = (
    temps_total_cumul - duree_cumul) / (freq_cumul * 24)


In [53]:
MTBF_MTTR_cat_cumul = MTBF_MTTR_cat_cumul[['duree_y', 'freq_y' ,'MTTR en h' ,'MTBF en j']].reset_index().rename(columns={'duree_y': 'duree', 'freq_y': 'freq'})

In [54]:
MTBF_MTTR_cat_cumul

,Catégorie,duree,freq,MTTR en h,MTBF en j
0,Brake,419.710556,41.0,10.236843,779.183221
1,Controller,1960.150833,170.0,11.530299,187.543100
2,Converter,1548.103054,150.0,10.320687,212.663305
3,Environement,14.952778,3.0,4.984259,10654.458989
4,Gear,747.715834,70.0,10.681655,456.183502
5,Generator,315.453056,10.0,31.545306,3195.085612
6,Grid,1704.545281,212.0,8.040308,150.438572
7,Hub,624.111389,55.0,11.347480,580.690825
8,Hydraulics,2038.000278,205.0,9.941465,155.507724
9,MISCELLANEOUS,83.502501,7.0,11.928929,4565.788676


In [55]:
df_ax4 = MTBF_MTTR_cat_cumul[['MTTR en h', 'MTBF en j']].reset_index()

# df_ax4.rename(columns={'MTTR en h': 'MTTR', 'MTBF en j': 'MTBF'}, inplace=True)

In [56]:
df_ax4

,index,MTTR en h,MTBF en j
0,0,10.236843,779.183221
1,1,11.530299,187.543100
2,2,10.320687,212.663305
3,3,4.984259,10654.458989
4,4,10.681655,456.183502
5,5,31.545306,3195.085612
6,6,8.040308,150.438572
7,7,11.347480,580.690825
8,8,9.941465,155.507724
9,9,11.928929,4565.788676


## Graphe 5 ax8

In [57]:
df_ax8 = pd.merge(
        (cumul_results.groupby('StationId')
                       .agg(**{'Durée alarmes': ('Duration 115(s)', lambda x: x.sum()/3600),
                               'Durée autres': ('Duration 20-25(s)', lambda x: x.sum()/3600)})
                       .reset_index()
         ),
         (cumul_alarms.groupby('StationId')
                      .agg(**{'Freq alarmes': ('Alarmcode', lambda x: int(x[x == 115].count()/2)),
                              'Freq autres': ('Alarmcode', lambda x: x[x == 20].count())})
                      .reset_index()
         ),
        ).sort_values('Durée alarmes', ascending=False).head(20)

# df_ax8.plot(kind='bar', x='StationId', ax=ax8)

# for stacked bar excel chart
df_ax8['Duration_20_25 reel'] = df_ax8['Durée autres']
df_ax8['Durée autres'] = df_ax8['Duration_20_25 reel'] - df_ax8['Durée alarmes']

df_ax8

,StationId,Durée alarmes,Durée autres,Freq alarmes,Freq autres,Duration_20_25 reel
68,69,852.326667,320.897778,64,205,1173.224444
40,41,420.651944,344.513056,95,273,765.165000
33,34,415.475556,344.119167,129,272,759.594722
82,83,347.886389,399.963333,85,313,747.849722
65,66,341.645000,380.488889,173,310,722.133889
107,108,336.062222,495.928056,236,389,831.990278
100,101,330.964722,414.197222,149,346,745.161944
74,75,314.727500,372.348611,167,320,687.076111
128,129,303.789444,451.873889,91,370,755.663333
6,7,296.696944,346.343056,185,323,643.040000


## str ax12

In [58]:
ELNX_cumul = round(cumul_results.ELNX.sum() / 10e5, 2)
ELNX_period = round(results.ELNX.sum() / 10e5, 2)


In [59]:
Ep_cumul = cumul_results['wtc_kWG1TotE_accum'].sum()
EL_cumul = cumul_results['EL'].sum()
ELX_cumul = cumul_results['ELX'].sum()
ELNX_cumul = cumul_results['ELNX'].sum()
Epot_cumul = cumul_results['Epot'].sum()

EL_wind_cumul = cumul_results['EL_wind'].sum()
EL_wind_start_cumul = cumul_results['EL_wind_start'].sum()
EL_alarm_start_cumul = cumul_results['EL_alarm_start'].sum()

MAA_cumul = round(100 * (
    Ep_cumul + ELX_cumul) / (
        Ep_cumul + EL_cumul - (EL_wind_cumul + EL_wind_start_cumul + EL_alarm_start_cumul)), 2)

In [60]:
Ep_period = results['wtc_kWG1TotE_accum'].sum()
EL_period = results['EL'].sum()
ELX_period = results['ELX'].sum()
ELNX_period = results['ELNX'].sum()
Epot_period = results['Epot'].sum()

EL_wind_period = results['EL_wind'].sum()
EL_wind_start_period = results['EL_wind_start'].sum()
EL_alarm_start_period = results['EL_alarm_start'].sum()

MAA_period = round(100 * (
    Ep_period + ELX_period) / (
        Ep_period + EL_period - (EL_wind_period + EL_wind_start_period + EL_alarm_start_period)), 2)

In [61]:
MAA_period

98.34

In [62]:
str_ax12 = f"""ELNX imputée à SGRE:  {ELNX_cumul} GWh ({ELNX_period} GWh pour {period_month_name} {period_year})
Disponibilité en énergie : {MAA_cumul}% ({MAA_period} % pour {period_month_name} {period_year})"""

In [63]:
str_ax12

'ELNX imputée à SGRE:  20618346.557225678 GWh (2449841.5054551647 GWh pour Août 2020)\nDisponibilité en énergie : 97.55% (98.34 % pour Août 2020)'

## Graph 7 ax18

In [ ]:
df_ax18 = (cumul_results[['StationId','ELNX', 'EL_indefini_left']]
                .groupby('StationId')
                .sum()
                .sort_values('StationId')
                # .head(20)
                .reset_index())

# Export XLSX writer

In [64]:
writer = pd.ExcelWriter(f'./input/{period}/output_xlsxwriter.xlsx')
# -------------------------------------------------------------------------------------------

workbook = writer.book
dashsheet = workbook.add_worksheet('Dash')

# -------------------------------------------------------------------------------------------

MTBF_MTTR_cat_cumul.to_excel(writer, sheet_name='MTBF & MTTR cumul', index=False)

# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax1-------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax1.to_excel(writer, sheet_name='ax1')

# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax3-------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax3.to_excel(writer, index=False, sheet_name='ax3')

worksheet = writer.sheets['ax3']


def make_chart_ax3():
    column_chart = workbook.add_chart({'type': 'column'})

    column_chart.add_series({'name': ['ax3', 0, 2],
                             'categories': ['ax3', 1, 0, 13, 0],
                             'values': ['ax3', 1, 2, 13, 2], })

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax3', 1, 1, 13, 1],
        'categories': ['ax3', 1, 0, 13, 0],
        'name': ['ax3', 0, 1],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    # column_chart.set_x_axis({'name': df_ax3.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': 'Cumul annuel par type d\'alarme',
                            'name_font': {'size': 12, 'bold': True}})
    line_chart.set_y2_axis({'name': df_ax3.columns[1]})
    return column_chart


column_chart = make_chart_ax3()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax3()
# Insert the chart into the worksheet.
dashsheet.insert_chart('B2', column_chart)


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax4-------------------------------------------------
# -------------------------------------------------------------------------------------------
df_ax4.to_excel(writer, index=False, sheet_name='ax4')
worksheet = writer.sheets['ax4']


def make_chart_ax4():
    column_chart = workbook.add_chart({'type': 'column'})

    column_chart.add_series({'name': ['ax4', 0, 2],
                             'categories': ['ax4', 1, 0, 13, 0],
                             'values': ['ax4', 1, 2, 13, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax4', 1, 1, 13, 1],
        'categories': ['ax4', 1, 0, 13, 0],
        'name': ['ax4', 0, 1],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)
    # Configure the chart axes.
    # column_chart.set_x_axis({'name': df_ax4.columns[0]})
    column_chart.set_y_axis({'name': 'MTBF en j'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': 'MTBF et MTTR par catégorie sur l\'année 2020',
                            'name_font': {'size': 12, 'bold': True}})
    line_chart.set_y2_axis({'name': 'MTTR en h'})
    return column_chart


column_chart = make_chart_ax4()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax4()
# Insert the chart into the worksheet.
dashsheet.insert_chart('J2', column_chart)



# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax5-------------------------------------------------
# -------------------------------------------------------------------------------------------
df_ax5.to_excel(writer, index=False, sheet_name='ax5')

worksheet = writer.sheets['ax5']


def make_chart_ax5():
    column_chart = workbook.add_chart({'type': 'column'})

    column_chart.add_series({'name': ['ax5', 0, 2],
                             'categories': ['ax5', 1, 0, 13, 0],
                             'values': ['ax5', 1, 2, 13, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax5', 1, 1, 13, 1],
        'categories': ['ax5', 1, 0, 13, 0],
        'name': ['ax5', 0, 1],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    # column_chart.set_x_axis({'name': df_ax5.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': f'Type d\'alarme {period}',
                            'name_font': {'size': 12, 'bold': True}})

    line_chart.set_y2_axis({'name': df_ax5.columns[1]})
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax5()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax5()
# Insert the chart into the worksheet.
dashsheet.insert_chart('R2', column_chart)


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax6-------------------------------------------------
# -------------------------------------------------------------------------------------------
df_ax6.to_excel(writer, index=False, sheet_name='ax6')

worksheet = writer.sheets['ax6']


def make_chart_ax6():
    column_chart = workbook.add_chart({'type': 'column'})

    column_chart.add_series({'name': ['ax6', 0, 2],
                             'categories': ['ax6', 1, 0, 20, 0],
                             'values': ['ax6', 1, 2, 20, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax6', 1, 1, 20, 1],
        'categories': ['ax6', 1, 0, 20, 0],
        'name': ['ax6', 0, 1],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax6.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': f'Alarmes {period}',
                            'name_font': {'size': 12, 'bold': True}})

    line_chart.set_y2_axis({'name': df_ax6.columns[1]})
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax6()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax6()
# Insert the chart into the worksheet.
dashsheet.insert_chart('Z2', column_chart)

# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax7-------------------------------------------------
# -------------------------------------------------------------------------------------------

worksheet = workbook.add_worksheet('ax7')

options = {
    'x_scale': 3,
    'y_scale': 25,
}

worksheet.insert_textbox(0, 0, ax7, options)



# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax8-------------------------------------------------
# -------------------------------------------------------------------------------------------
df_ax8.to_excel(writer, index=False, sheet_name='ax8')

worksheet = writer.sheets['ax8']


def make_chart_ax8():
    column_chart = workbook.add_chart({'type': 'column',
                                       'subtype': 'stacked'})

    column_chart.add_series({'name': ['ax8', 0, 1],
                             'categories': ['ax8', 1, 0, 20, 0],
                             'values': ['ax8', 1, 1, 20, 1]})

    column_chart.add_series({'name': ['ax8', 0, 2],
                             'categories': ['ax8', 1, 0, 20, 0],
                             'values': ['ax8', 1, 2, 20, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax8', 1, 3, 20, 3],
        'categories': ['ax8', 1, 0, 20, 0],
        'name': ['ax8', 0, 3],
        'y2_axis': True,
    })

    line_chart.add_series({
        'values': ['ax8', 1, 4, 20, 4],
        'categories': ['ax8', 1, 0, 20, 0],
        'name': ['ax8', 0, 4],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax8.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': 'Arrêts turbines : Cumul Annuel',
                            'name_font': {'size': 12, 'bold': True}})

    line_chart.set_y2_axis({'name': 'Freq'})
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax8()
# Insert the chart into the worksheet.
worksheet.insert_chart('G2', column_chart)

column_chart = make_chart_ax8()
# Insert the chart into the worksheet.
dashsheet.insert_chart('B17', column_chart)


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax9-------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax9.to_excel(writer, index=False, sheet_name='ax9')

worksheet = writer.sheets['ax9']


def make_chart_ax9():
    column_chart = workbook.add_chart({'type': 'column',
                                       'subtype': 'stacked'})

    column_chart.add_series({'name': ['ax9', 0, 1],
                             'categories': ['ax9', 1, 0, 20, 0],
                             'values': ['ax9', 1, 1, 20, 1]})

    column_chart.add_series({'name': ['ax9', 0, 2],
                             'categories': ['ax9', 1, 0, 20, 0],
                             'values': ['ax9', 1, 2, 20, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax9', 1, 3, 20, 3],
        'categories': ['ax9', 1, 0, 20, 0],
        'name': ['ax9', 0, 3],
        'y2_axis': True,
    })

    line_chart.add_series({
        'values': ['ax9', 1, 4, 20, 4],
        'categories': ['ax9', 1, 0, 20, 0],
        'name': ['ax9', 0, 4],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax9.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': f'Arrêts turbines {period}',
                            'name_font': {'size': 12, 'bold': True}})

    line_chart.set_y2_axis({'name': 'Freq'})
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax9()
# Insert the chart into the worksheet.
worksheet.insert_chart('G2', column_chart)

column_chart = make_chart_ax9()
# Insert the chart into the worksheet.
dashsheet.insert_chart('J17', column_chart)


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax10------------------------------------------------
# -------------------------------------------------------------------------------------------
df_ax10.to_excel(writer, index=False, sheet_name='ax10')

worksheet = writer.sheets['ax10']


def make_chart_ax10():
    column_chart = workbook.add_chart({'type': 'column',
                                       'subtype': 'stacked'})

    for col in range(1, 3):
        column_chart.add_series({'values': ['ax10', 1, col, 20, col],
                                 'categories': ['ax10', 1, 0, 20, 0],
                                 'name': ['ax10', 0, col]})

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax10.columns[0]})
    # column_chart.set_y_axis({'name': df_ax10.columns[1]})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title(
        {'name': f'Energie perdue selon FSA du {period}',
         'name_font': {'size': 12, 'bold': True}}
    )
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax10()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax10()
# Insert the chart into the worksheet.
dashsheet.insert_chart('R17', column_chart)


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax11------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax11.to_excel(writer, index=False, sheet_name='ax11')


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax12------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax12.to_excel(writer, sheet_name='ax12')

# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax18------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax18.to_excel(writer, index=False, sheet_name='ax18')

worksheet = writer.sheets['ax18']


def make_chart_ax18():
    column_chart = workbook.add_chart({'type': 'column',
                                       'subtype': 'stacked'})

    for col in range(1, 3):
        column_chart.add_series({'values': ['ax18', 1, col, 131, col],
                                 'categories': ['ax18', 1, 0, 131, 0],
                                 'name': ['ax18', 0, col]})

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax18.columns[0]})
    # column_chart.set_y_axis({'name': df_ax18.columns[1]})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title(
        {'name': 'Energie perdue selon FSA cumulée sur l\'année 2020 en MWh',
         'name_font': {'size': 12, 'bold': True}}
    )
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax18()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax18()
# Insert the chart into the worksheet.
dashsheet.insert_chart('B32', column_chart, {'x_scale': 4, 'y_scale': 1})

# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax19------------------------------------------------
# -------------------------------------------------------------------------------------------
# df_ax19.to_excel(writer, index=False, sheet_name='ax19')
# 
# worksheet = writer.sheets['ax19']
# 
# 
# def make_chart_ax19():
#     column_chart = workbook.add_chart({'type': 'column'})
# 
#     column_chart.add_series({'values': ['ax19', 1, 1, 131, 1],
#                              'categories': ['ax19', 1, 0, 131, 0],
#                              'name': ['ax19', 0, 1]})
# 
#     # Configure the chart axes.
#     column_chart.set_x_axis({'name': df_ax19.columns[0]})
#     # column_chart.set_y_axis({'name': df_ax19.columns[1]})
#     column_chart.set_legend({'position': 'bottom'})
#     column_chart.set_title(
#         {'name': f'Ratio Courbe Théorique vs Courbe Réelle du {period}',
#          'name_font': {'size': 12, 'bold': True}}
#     )
#     return column_chart
# 
# 
# # Insert the chart into the worksheet.
# column_chart = make_chart_ax19()
# # Insert the chart into the worksheet.
# worksheet.insert_chart('E2', column_chart)
# 
# column_chart = make_chart_ax19()
# # Insert the chart into the worksheet.
# dashsheet.insert_chart('B47', column_chart, {'x_scale': 4, 'y_scale': 1})


writer.save()

NameError: name 'df_ax18' is not defined

In [ ]:
# !jupyter nbconvert performance.ipynb --to markdown